In [64]:
import featuretools as ft
import pandas as pd

# df = pd.DataFrame({'A': [1, 1, 2, 2],
#                    'B': [1, 2, 3, 4],
#                    'C': [0.3, 0.2, 1.2, -0.5]})

# es = ft.EntitySet()

# es.entity_from_dataframe(entity_id="example",
#                           index="id",
#                           make_index=True,
#                           dataframe=df)

# es.normalize_entity(new_entity_id="a_entity",
#                     base_entity_id="example",
#                     index="A")

# fm, fl = ft.dfs(target_entity="example",
#                 entityset=es,
#                 agg_primitives=["min"])

# fm


In [2]:
from mlbt.load_data import load_bars

/Users/doda/anaconda3/envs/metal/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.bagging module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/doda/anaconda3/envs/metal/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)


In [69]:
es = load_bars('@ES#C', {'bar_type': 'dollar', 'load_from_disk': True}).reset_index(drop=True)
nq = load_bars('@NQ#C', {'bar_type': 'dollar', 'load_from_disk': True}).reset_index(drop=True)

In [70]:
es['Symbol'] = '@ES#C'
nq['Symbol'] = '@NQ#C'

In [71]:
# df = pd.concat([es, nq])
df = es

In [72]:
# df['SectorId'] = 1

In [73]:
# del df['sector_id']

In [74]:
            feat = engineer_feature(deck, symbol, config, feat_config.copy())["Close"]


,Time,Open,High,Low,Close,Volume,Dollar Volume,Num Ticks,Buy Volume,Symbol
0,2005-09-07 10:16:00,1178.50,1179.25,1175.75,1175.75,133923,1.576325e+08,685,58250,@ES#C
1,2005-09-07 11:18:00,1175.75,1179.75,1175.75,1177.75,136639,1.609713e+08,62,72287,@ES#C
2,2005-09-07 14:07:00,1178.00,1180.75,1177.00,1180.25,133040,1.568281e+08,169,73569,@ES#C
3,2005-09-07 15:23:00,1180.00,1181.50,1177.00,1181.25,134170,1.582662e+08,76,65600,@ES#C
4,2005-09-08 09:35:00,1181.25,1181.50,1169.50,1176.75,134103,1.580449e+08,703,57787,@ES#C
...,...,...,...,...,...,...,...,...,...,...
58382,2020-01-16 09:31:00,3303.75,3305.75,3303.25,3304.75,51687,1.708143e+08,65,20683,@ES#C
58383,2020-01-16 09:37:00,3304.75,3307.75,3304.75,3306.00,54179,1.791396e+08,6,27622,@ES#C
58384,2020-01-16 09:46:00,3306.25,3308.00,3306.25,3307.25,51165,1.692045e+08,9,43824,@ES#C
58385,2020-01-16 09:53:00,3305.00,3305.00,3304.00,3304.25,53755,1.776221e+08,7,19061,@ES#C


In [75]:

es = ft.EntitySet()

es.entity_from_dataframe(entity_id="bars",
                          index="id",
#                           make_index=True,
                          dataframe=df)


2020-02-09 17:56:51,688 featuretools.entityset - WARNING    index id not found in dataframe, creating new integer column


Entityset: None
  Entities:
    bars [Rows: 58387, Columns: 11]
  Relationships:
    No relationships

In [77]:
from featuretools.primitives import make_agg_primitive, make_trans_primitive

from featuretools.variable_types import Text, Numeric



In [89]:

def ma(column, window=20):
    return column.rolling(window).mean()

def ma_generate_name(self, base_feature_names):
    return u'MA(%s, "%s")' % (base_feature_names[0], self.kwargs['window'])    

MA = make_trans_primitive(function=ma,
                                   input_types=[Numeric],
                                   return_type=Numeric,
                                   cls_attributes={"generate_name": ma_generate_name})
   
def volratio(column, window=20):
    return column.rolling(window).mean()

def volratio_generate_name(self, base_feature_names):
    return u'VolRatio(%s, "%s")' % (base_feature_names[0], self.kwargs['window'])    

MA = make_trans_primitive(function=volratio,
                                   input_types=[Numeric, Numeric],
                                   return_type=Numeric,
                                   cls_attributes={"generate_name": volratio_generate_name})
   

In [86]:

agg_primitives=[
        'std', 'min', 'max', 'mean', 
#         'percent_true', 'last', 'count', 
#         'trend', 'n_most_common', 'time_since_last',        
#         'avg_time_between'
]
trans_primitives=[
#         'year', 'month', 'weekday', 'percentile', 
#         'isin', 'cum_mean',
#         'time_since_previous',
        MA(window=30),
        MA(window=200),    
]

fm, fl = ft.dfs(target_entity="bars",
                entityset=es,
                agg_primitives=agg_primitives,
                trans_primitives=trans_primitives,
                max_depth=1,
)

fm

,Open,High,Low,Close,Volume,Dollar Volume,Num Ticks,Buy Volume,Symbol,"MA(Low, ""30"")",...,"MA(Dollar Volume, ""30"")","MA(Volume, ""30"")","MA(Low, ""200"")","MA(Open, ""200"")","MA(Close, ""200"")","MA(High, ""200"")","MA(Buy Volume, ""200"")","MA(Num Ticks, ""200"")","MA(Dollar Volume, ""200"")","MA(Volume, ""200"")"
id,,,,,,,,,,,,,,,,,,,,,
0,1178.50,1179.25,1175.75,1175.75,133923,1.576325e+08,685,58250,@ES#C,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1175.75,1179.75,1175.75,1177.75,136639,1.609713e+08,62,72287,@ES#C,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1178.00,1180.75,1177.00,1180.25,133040,1.568281e+08,169,73569,@ES#C,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1180.00,1181.50,1177.00,1181.25,134170,1.582662e+08,76,65600,@ES#C,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1181.25,1181.50,1169.50,1176.75,134103,1.580449e+08,703,57787,@ES#C,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58382,3303.75,3305.75,3303.25,3304.75,51687,1.708143e+08,65,20683,@ES#C,3291.433333,...,1.623820e+08,49308.966667,3266.10375,3268.2750,3268.67000,3270.63875,25965.380,47.755,1.647102e+08,50399.505
58383,3304.75,3307.75,3304.75,3306.00,54179,1.791396e+08,6,27622,@ES#C,3292.283333,...,1.628219e+08,49429.700000,3266.43625,3268.6075,3269.00875,3270.97000,25956.205,47.755,1.647086e+08,50393.470
58384,3306.25,3308.00,3306.25,3307.25,51165,1.692045e+08,9,43824,@ES#C,3292.933333,...,1.631348e+08,49515.133333,3266.80125,3268.9600,3269.37875,3271.33125,26139.180,47.770,1.647015e+08,50385.560


In [87]:
fm.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Dollar Volume', 'Num Ticks',
       'Buy Volume', 'Symbol', 'MA(Low, "30")', 'MA(Open, "30")',
       'MA(Close, "30")', 'MA(High, "30")', 'MA(Buy Volume, "30")',
       'MA(Num Ticks, "30")', 'MA(Dollar Volume, "30")', 'MA(Volume, "30")',
       'MA(Low, "200")', 'MA(Open, "200")', 'MA(Close, "200")',
       'MA(High, "200")', 'MA(Buy Volume, "200")', 'MA(Num Ticks, "200")',
       'MA(Dollar Volume, "200")', 'MA(Volume, "200")'],
      dtype='object')

In [88]:
fm[['MA(Close, "30")', 'Close']]

,"MA(Close, ""30"")",Close
id,,
0,NaN,1175.75
1,NaN,1177.75
2,NaN,1180.25
3,NaN,1181.25
4,NaN,1176.75
...,...,...
58382,3293.800000,3304.75
58383,3294.491667,3306.00
58384,3295.066667,3307.25
